## Likelihood fit to exponential data

This notebook shows how to perform a likelihood fit to a data sample distributed according to an exponential distribution.
The data sample consiss of n events (e.g. n=20) generated randomly using the ROOT exponential random number generator. 
A negative log-likelihood function is constructed using the given data set. The log-likelihood function is a function of the fit parameter `tau`, which is the slope of the exponential distribution and the minimum of the function is found using simple tools provided by ROOT.  
The example shows also how to extimate the error of the parameter using both the second derivative of the log-likelihood function or the `DeltaL=0.5` rule. 
By increasing the number of generated events one can see how the likelihood shape approaches a parabola and the errors estimated in the two methods become equal. 


#### 1. Event generation
First we generate n events using an exponential distribution with mean life tau=2

In [1]:
int n = 20;
double tau = 2; 
std::vector<double> x(n);
TRandom3 r(0);
for (int i = 0; i < n; ++i) 
    x[i] = r.Exp(tau)

In [2]:
// comment this line for avoiding printing the random generated values 
x

(std::vector<double> &) { 5.34268, 2.20038, 1.03052, 0.421726, 2.54175, 0.143218, 0.915990, 1.63747, 0.129059, 8.72879, 0.774927, 0.653277, 1.83702, 1.74850, 0.908466, 2.49153, 0.193633, 1.56319, 0.177877, 2.83221 }


#### 2. Make an histogram of the generated data

We create a ROOT histogram and we fill with the content of the data vector we have created before. 
Note that to display the histogram we use the javascript graphics option (*jsroot on*)

In [3]:
%jsroot on

In [4]:
auto h1 = new TH1D("h1","Generated Exponential data",10,0,5*tau);
for (auto x_i : x)  h1->Fill(x_i);
h1->Draw();
gPad->Draw();

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


#### 3. Calculation of the Negative Log-Likelihood function
We compute now directly the negative log-likelihood function. We assume the data are distributed according to an exponential distribution. 
We need to compute the sum of the log of the pdf(x). The pdf is 
$$ PDF(x | \tau) = \frac{1}{\tau} e ^{- {x / \tau} } $$
The negative log-likelihood is then 
$$ NLL ( x_i, \tau ) =  -  \sum_{i-events} PDF(x_i | \tau) = \sum_{i-events} \log \frac{1}{\tau} e ^{- {x_i / \tau} } $$

In [5]:
auto nll = [&](double * p, double *){ double sum = 0; double tau = p[0]; 
                           for (auto x_i : x) {
                               sum += std::log(1./tau * std::exp(-x_i/tau));
                           }
                           return -sum; 
                          };

#### 4. Plot of the negative log-likelihood function 
Plot now the likelihood as function of the exponential parameter (tau)

In [6]:
fnll = new TF1("nll",nll,.1,10,0);

In [7]:
fnll->SetRange(1,5);
fnll->SetTitle("Negative Log-Likelihood function;tau;NLL");
fnll->Draw();
gPad->Draw();

#### 5. Find the minimum of the NLL

We search now for the minimum of the negative log-likelihood function. We are interested in the value of the parameter tau for which the negative log-likelihood function has a minimum.
We use the function provided by ROOT (in the **TF1** class) to find the function minimum.

In [8]:
double tau_hat = fnll->GetMinimumX();
std::cout << "Estimated value of tau = " << tau_hat << std::endl;

Estimated value of tau = 1.81361


#### 6. Estimation of the fit parameter uncertainty

We are interested now in estimating the uncertainty on the est fit parameter. 
We can estimate this uncertainty in two way.

##### 6a. Using the second derivative (Hesse method)
The first one is by using the second derivatives of the log-likelihood function. We compute then the second derivative of the function at the minimum. This method assumes that the negative log-likelihood function is a parabola around its minimum and the computed interval is symmetric round the best parameter value. 

In [9]:
double h = fnll->Derivative2(tau_hat);

The error can be computed then from the square root of inverse of the second derivative. Note that in case of more than one fitting the parameter the covariance matrix is obtained from the inverse of the Hessian (second derivatives) matrix.  

In [10]:
double error = sqrt(1./h);
std::cout << "Parabolic error on tau = " << error << std::endl; 

Parabolic error on tau = 0.405535


##### 6b Using the negative log-likelihood function values (MINOS method) 
With the second method, we estimate the error using the DeltaL = 0.5 rule. We look at the parameter values for which the negative log-likelihood has a value equal to the minimum value + 0.5. 
The errors computed with this method are asymmetric. in case of a multi-dimensional log-likelihood function a program exists to compute them (MINOS). 

In [11]:
double nll_min = fnll->Eval(tau_hat); 
std::cout << "nll function value at minimum is = " << nll_min << std::endl;

nll function value at minimum is = 31.9064


In [12]:
double tau_low = fnll->GetX(nll_min+0.5,1,tau_hat);

In [13]:
double tau_high = fnll->GetX(nll_min+0.5,tau_hat,10);

In [14]:
std::cout << "The interval for tau is :  [ " << tau_low << " , " << tau_high << " ] " << std::endl;
double err_low = tau_hat - tau_low;
double err_high = tau_high - tau_hat; 
std::cout << "Best estimate of tau is : " << tau_hat << " - " << err_low << " + " << err_high << std::endl;

The interval for tau is :  [ 1.46191 , 2.28777 ] 
Best estimate of tau is : 1.81361 - 0.351699 + 0.474164


Make a plot of the likelihood around the interval. 

In [15]:
// Use a +/- 2 sigma interval for the plot
fnll->SetRange(tau_hat-2*(tau_hat-tau_low),tau_hat+2*(tau_high-tau_hat));
fnll->SetMinimum(nll_min);
fnll->Draw();
l1 = new TLine(tau_low,nll_min,tau_low,nll_min+0.5);
l2 = new TLine(tau_high,nll_min,tau_high,nll_min+0.5);
l1->SetLineColor(kBlue); l2->SetLineColor(kBlue); 
l1->SetLineWidth(3); l2->SetLineWidth(3);
l1->Draw();
l2->Draw();
gPad->Draw();

### Binned likelihood fit

As comparison we perform a binned likelihood fit to the histogram containing the exponential data.
When fitting an histogram in ROOT we have the following methods available
   -  Binned Maximum Likelihood Fit (option **L**). The likelihood is build assuming a Poisson p.d.f. in each bin. The Likelihood ratio proposed by *Baker and Cousins, NIM A221 (1984) 437* is computed.
   -  Neyman Chi-square (default). This method uses the observed errors as weights in the least square sum. Empty bins are excluded from the fit
   -  Pearson Chi-square (option **P** ). This method uses the expected errors. 
   
We use also the fit option "S" to save the fit result and print it.

We will see tomorrow the Chi-Squared methods. Now we perform the fit using the likelihood method. 

In [16]:
f1 = new TF1("f1","[A]*exp(-x/[tau])");
f1->SetParameters(1,1);
//ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");  // to use Minuit2 as defult minimizer
fitResult = h1->Fit(f1,"L S");
fitResult->Print();

 FCN=4.87427 FROM MIGRAD    STATUS=CONVERGED      80 CALLS          81 TOTAL
                     EDM=8.714e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            1.13519e+01   3.74691e+00   4.05849e-03   2.33106e-06
   2  tau          1.79154e+00   4.32924e-01   4.69732e-04   8.16425e-05
                               ERR DEF= 0.5

****************************************
Minimizer is Minuit / Migrad
MinFCN                    =      4.87427
Chi2                      =      4.97069
NDf                       =            8
Edm                       =    8.714e-10
NCalls                    =           81
A                         =      11.3519   +/-   3.74691     
tau                       =      1.79154   +/-   0.432924    


In [17]:
gStyle->SetOptFit(111);
gPad->Draw()

#### Bias study of fit parameter

We can study the bias of the fitted parameter, by generating a series of pseudo-experiments, fitting each one of them and plot the obtained parameter distribution

In [18]:
auto htau = new TH1D("htau","Distribution of fitted parameter tau",50,0,5);

In [19]:
htau->Reset();  // in case we run more than one time
nexperiments = 1000;
for (int iexp = 0; iexp < nexperiments; ++iexp) {
    h1->Reset();
    for (int i = 0; i < n; ++i) {         
        h1->Fill(r.Exp(tau) ); 
    } 
    f1->SetParameters(1,1);
    h1->Fit(f1,"L Q"); // use option Q to avoid too much printout
    double tau_fit = f1->GetParameter("tau");
    htau->Fill(tau_fit);
}

In [20]:
htau->Fit("gaus");
gPad->Draw();

 FCN=144.915 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=9.90941e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.14192e+01   3.17063e+00   1.44052e-02  -2.98615e-06
   2  Mean         2.01917e+00   1.76964e-02   9.88929e-05   1.72269e-03
   3  Sigma        4.80284e-01   1.40638e-02   3.94325e-05  -5.45078e-03


Optionally one can study also the same bias using the unbinned maximum-likelihood fit and the chi-squared fit